# This code is based on suggestions of Dr. James from LICOR on how to calibrate the latent heat flux
## The code will use the calibration values obtained from the field and the instrument's factory calibration values to correct the latent heat flux values in the 20 Hz data files


#### Input directory: D : \EC_GHG\2020-2021\Summer\Way3 HorstCorr
#### Output directory : D:\EC_GHG\2020-2021\Summer\MariaLEFix\OutputDataFromLE_Fix_Dr_James

#### Folder information
    Folder size: 105 GB
    Number of files: 7221
    File size average: 15000 KB

#### Formulas 
    (𝑇𝑟𝑎𝑛𝑠𝑚𝑖𝑡𝑡𝑎𝑛𝑐𝑒)_1 = 1 /(𝐻_2 𝑂 𝑍𝑒𝑟𝑜)
    (𝑇𝑟𝑎𝑛𝑠𝑚𝑖𝑡𝑡𝑎𝑛𝑐𝑒)_𝑛 = (𝑇𝑟𝑎𝑛𝑠𝑚𝑖𝑡𝑡𝑎𝑛𝑐𝑒)_(𝑛−1) − 0.01
     𝑋 𝑖𝑛 𝑝𝑜𝑙𝑦𝑛𝑜𝑚𝑖𝑎𝑙 = [(1−𝑇𝑟𝑎𝑛𝑠𝑚𝑖𝑡𝑡𝑎𝑛𝑐𝑒)(𝐻_2 𝑂 𝑍𝑒𝑟𝑜)][(𝐻_2 𝑂 𝑆𝑝𝑎𝑛)/𝑃𝑟𝑒𝑠𝑠𝑢𝑟𝑒]
     𝐷𝑒𝑛𝑠𝑖𝑡𝑦 = [(𝐴)(𝑋 𝑖𝑛 𝑝𝑜𝑙𝑦𝑛𝑜𝑚𝑖𝑎𝑙)+(𝐵)(𝑋 𝑖𝑛 𝑝𝑜𝑙𝑦𝑛𝑜𝑚𝑖𝑎𝑙)^2+(𝐶) (𝑋 𝑖𝑛 𝑝𝑜𝑙𝑦𝑛𝑜𝑚𝑖𝑎𝑙)^3 ][𝑃𝑟𝑒𝑠𝑠𝑢𝑟𝑒]


In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from scipy import stats 
import matplotlib.pyplot as plt
from numpy import nan
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import matplotlib as matplotlibZ
import matplotlib.lines as mlines
from scipy import stats
from sympy import S, symbols, printing
import matplotlib
import os
import sklearn.metrics as metrics

### Storaging Field Values

In [3]:
H2OZero = 0.896865
H2OSpan = 0.353353
A = 5108.30
B = 3967550
C = -281375000

### Storaging Factory Values

In [4]:
H2OZero_F = 0.8870
H2OSpan_F = 1.0063
A_F = 5108.30
B_F = 3967550
C_F = -281375000

In [5]:
import os

In [6]:
input_directory = r'D:\EC_GHG\2020-2021\Summer\Way3 HorstCorr'

In [7]:
input_directory_Way4 = r'E:\EC_GHG\2020-2021\Summer\Way4 HorstCorr'

# Calculate the transmittance from incorrect molar density 

Use transmittance and factory corrected coefficients to calculate factory corrected molar density

Plot factory corrected molar density vs incorrected molar density to find slope and intercept 

Use the slope and intercept to correct the incorrected molar density 

In [5]:
import pandas as pd

import os

 

# Path to the directory containing the CSV files

directory = r'C:\Users\mv018\Documents\Files_for_Code\way3'

 

# Initialize an empty list to store individual DataFrames

dfs = []

 

# Iterate over each file in the directory

for filename in os.listdir(directory):

    if filename.endswith('.csv'):  # Consider only CSV files

        file_path = os.path.join(directory, filename)

        df = pd.read_csv(file_path, header = None)

        dfs.append(df)

 

# Concatenate all DataFrames into a single DataFrame

way3 = pd.concat(dfs, ignore_index=True)

 

# Print the combined DataFrame

print(way3)


          0           1          2           3    4   5           6   \
0       DATA  1632142800          0  3934696752  255   1  2021-09-20   
1       DATA  1632142800   50000000  3934696767  255   1  2021-09-20   
2       DATA  1632142800  100000000  3934696782  255   1  2021-09-20   
3       DATA  1632142800  150000000  3934696797  255   1  2021-09-20   
4       DATA  1632142800  200000000  3934696812  255   1  2021-09-20   
...      ...         ...        ...         ...  ...  ..         ...   
179995  DATA  1632151799  750000000  3937396677  255   1  2021-09-20   
179996  DATA  1632151799  800000000  3937396692  255   1  2021-09-20   
179997  DATA  1632151799  850000000  3937396707  255   1  2021-09-20   
179998  DATA  1632151799  900000000  3937396722  255   1  2021-09-20   
179999  DATA  1632151799  950000000  3937396737  255   1  2021-09-20   

                  7         8         9   ...        41       42       43  \
0       08:00:00:000  0.101424  0.110803  ...  0.000157  9

In [6]:
way3

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,DATA,1632142800,0,3934696752,255,1,2021-09-20,08:00:00:000,0.101424,0.110803,...,0.000157,9999.99,9999.99,9999.99,16399,0,250,0.000800,0.439116,0.143422
1,DATA,1632142800,50000000,3934696767,255,1,2021-09-20,08:00:00:050,0.101392,0.110796,...,0.000157,9999.99,9999.99,9999.99,16927,0,116,-0.001162,0.420922,0.127140
2,DATA,1632142800,100000000,3934696782,255,1,2021-09-20,08:00:00:100,0.101441,0.110833,...,0.000078,9999.99,9999.99,9999.99,16927,0,107,-0.024040,0.413913,0.092813
3,DATA,1632142800,150000000,3934696797,255,1,2021-09-20,08:00:00:150,0.101455,0.110768,...,0.000157,9999.99,9999.99,9999.99,16927,0,99,-0.019910,0.515804,0.089599
4,DATA,1632142800,200000000,3934696812,255,1,2021-09-20,08:00:00:200,0.101403,0.110896,...,0.000000,9999.99,9999.99,9999.99,16927,0,11,0.012735,0.532212,0.130540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632151799,750000000,3937396677,255,1,2021-09-20,10:29:59:750,0.088177,0.117902,...,0.000000,9999.99,9999.99,9999.99,15,0,246,1.354123,-0.486586,-0.338717
179996,DATA,1632151799,800000000,3937396692,255,1,2021-09-20,10:29:59:800,0.088175,0.117842,...,0.000078,9999.99,9999.99,9999.99,15,0,20,1.367132,-0.439551,-0.318392
179997,DATA,1632151799,850000000,3937396707,255,1,2021-09-20,10:29:59:850,0.088222,0.117746,...,0.000078,9999.99,9999.99,9999.99,15,0,210,1.606963,-0.408218,-0.295514
179998,DATA,1632151799,900000000,3937396722,255,1,2021-09-20,10:29:59:900,0.088219,0.117692,...,0.000000,9999.99,9999.99,9999.99,15,0,243,1.576140,-0.278051,-0.104778


In [8]:
import pandas as pd

import os

 

# Path to the directory containing the CSV files

directory = r'C:\Users\mv018\Documents\Files_for_Code\way4'

 

# Initialize an empty list to store individual DataFrames

dfs = []

 

# Iterate over each file in the directory

for filename in os.listdir(directory):

    if filename.endswith('.csv'):  # Consider only CSV files

        file_path = os.path.join(directory, filename)

        df = pd.read_csv(file_path, header = None)

        dfs.append(df)

 

# Concatenate all DataFrames into a single DataFrame

way4 = pd.concat(dfs, ignore_index=True)

 

# Print the combined DataFrame

print(way4)


          0           1          2           3    4   5           6   \
0       DATA  1632130200          0  3544178502  255   1  2021-09-20   
1       DATA  1632130200   50000000  3544178517  255   1  2021-09-20   
2       DATA  1632130200  100000000  3544178532  255   1  2021-09-20   
3       DATA  1632130200  150000000  3544178547  255   1  2021-09-20   
4       DATA  1632130200  200000000  3544178562  255   1  2021-09-20   
...      ...         ...        ...         ...  ...  ..         ...   
179995  DATA  1632139199  750000000  3546878427  255   1  2021-09-20   
179996  DATA  1632139199  800000000  3546878442  255   1  2021-09-20   
179997  DATA  1632139199  850000000  3546878457  255   1  2021-09-20   
179998  DATA  1632139199  900000000  3546878472  255   1  2021-09-20   
179999  DATA  1632139199  950000000  3546878487  255   1  2021-09-20   

                  7         8         9   ...       41       42       43  \
0       04:30:00:000  0.108009  0.100192  ...  9999.99  999

In [9]:
way4

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632139199,750000000,3546878427,255,1,2021-09-20,06:59:59:750,0.130402,0.098182,...,9999.99,9999.99,9999.99,15,0,23,NaN,0.074942,-0.260914,-0.016748
179996,DATA,1632139199,800000000,3546878442,255,1,2021-09-20,06:59:59:800,0.130391,0.098262,...,9999.99,9999.99,9999.99,15,0,242,NaN,0.092819,-0.245323,-0.012694
179997,DATA,1632139199,850000000,3546878457,255,1,2021-09-20,06:59:59:850,0.130510,0.098290,...,9999.99,9999.99,9999.99,15,0,241,NaN,0.075983,-0.260908,-0.016741
179998,DATA,1632139199,900000000,3546878472,255,1,2021-09-20,06:59:59:900,0.130490,0.098184,...,9999.99,9999.99,9999.99,15,0,85,NaN,0.076167,-0.258885,-0.012521


In [21]:
from datetime import datetime
way3["Datetime-merged"] = way3[6] + " " + way3[7]
way3["Datetime-merged"] = pd.to_datetime(way3["Datetime-merged"], format="%Y-%m-%d %H:%M:%S:%f")

In [22]:
way3

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Datetime-merged
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542,2021-09-20 04:30:00.000
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025,2021-09-20 04:30:00.050
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442,2021-09-20 04:30:00.100
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926,2021-09-20 04:30:00.150
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815,2021-09-20 04:30:00.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632139199,750000000,3546878427,255,1,2021-09-20,06:59:59:750,0.130402,0.098182,...,9999.99,9999.99,15,0,23,NaN,0.074942,-0.260914,-0.016748,2021-09-20 06:59:59.750
179996,DATA,1632139199,800000000,3546878442,255,1,2021-09-20,06:59:59:800,0.130391,0.098262,...,9999.99,9999.99,15,0,242,NaN,0.092819,-0.245323,-0.012694,2021-09-20 06:59:59.800
179997,DATA,1632139199,850000000,3546878457,255,1,2021-09-20,06:59:59:850,0.130510,0.098290,...,9999.99,9999.99,15,0,241,NaN,0.075983,-0.260908,-0.016741,2021-09-20 06:59:59.850
179998,DATA,1632139199,900000000,3546878472,255,1,2021-09-20,06:59:59:900,0.130490,0.098184,...,9999.99,9999.99,15,0,85,NaN,0.076167,-0.258885,-0.012521,2021-09-20 06:59:59.900


In [23]:
from datetime import datetime
way4["Datetime-merged"] = way4[6] + " " + way4[7]
way4["Datetime-merged"] = pd.to_datetime(way4["Datetime-merged"], format="%Y-%m-%d %H:%M:%S:%f")

In [24]:
way4

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Datetime-merged
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542,2021-09-20 04:30:00.000
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025,2021-09-20 04:30:00.050
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442,2021-09-20 04:30:00.100
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926,2021-09-20 04:30:00.150
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815,2021-09-20 04:30:00.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632139199,750000000,3546878427,255,1,2021-09-20,06:59:59:750,0.130402,0.098182,...,9999.99,9999.99,15,0,23,NaN,0.074942,-0.260914,-0.016748,2021-09-20 06:59:59.750
179996,DATA,1632139199,800000000,3546878442,255,1,2021-09-20,06:59:59:800,0.130391,0.098262,...,9999.99,9999.99,15,0,242,NaN,0.092819,-0.245323,-0.012694,2021-09-20 06:59:59.800
179997,DATA,1632139199,850000000,3546878457,255,1,2021-09-20,06:59:59:850,0.130510,0.098290,...,9999.99,9999.99,15,0,241,NaN,0.075983,-0.260908,-0.016741,2021-09-20 06:59:59.850
179998,DATA,1632139199,900000000,3546878472,255,1,2021-09-20,06:59:59:900,0.130490,0.098184,...,9999.99,9999.99,15,0,85,NaN,0.076167,-0.258885,-0.012521,2021-09-20 06:59:59.900


In [26]:
dfSmerged= way3.merge(way4, how='left', on='Datetime-merged')

dfSmerged

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,41_y,42_y,43_y,44_y,45_y,46_y,47_y,48_y,49_y,50_y
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632139199,750000000,3546878427,255,1,2021-09-20,06:59:59:750,0.130402,0.098182,...,9999.99,9999.99,9999.99,15,0,23,NaN,0.074942,-0.260914,-0.016748
179996,DATA,1632139199,800000000,3546878442,255,1,2021-09-20,06:59:59:800,0.130391,0.098262,...,9999.99,9999.99,9999.99,15,0,242,NaN,0.092819,-0.245323,-0.012694
179997,DATA,1632139199,850000000,3546878457,255,1,2021-09-20,06:59:59:850,0.130510,0.098290,...,9999.99,9999.99,9999.99,15,0,241,NaN,0.075983,-0.260908,-0.016741
179998,DATA,1632139199,900000000,3546878472,255,1,2021-09-20,06:59:59:900,0.130490,0.098184,...,9999.99,9999.99,9999.99,15,0,85,NaN,0.076167,-0.258885,-0.012521


In [27]:
dfSmerged["11_x"]

0          894.361
1          894.034
2          893.195
3          893.161
4          894.653
            ...   
179995    1158.780
179996    1158.670
179997    1160.160
179998    1159.910
179999    1158.290
Name: 11_x, Length: 180000, dtype: float64

In [28]:
dfSmerged["12_x"]

0         1055.38
1         1055.58
2         1056.15
3         1056.47
4         1055.23
           ...   
179995    1024.24
179996    1025.49
179997    1025.91
179998    1024.30
179999    1023.85
Name: 12_x, Length: 180000, dtype: float64

In [35]:
# Filter observations between 8 AM and 8 PM
way3_filtered = way3[(way3["Datetime-merged"].dt.hour >= 4) & (way3["Datetime-merged"].dt.hour < 6)]

In [36]:
way3_filtered

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Datetime-merged
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542,2021-09-20 04:30:00.000
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025,2021-09-20 04:30:00.050
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442,2021-09-20 04:30:00.100
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926,2021-09-20 04:30:00.150
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815,2021-09-20 04:30:00.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,DATA,1632135599,750000000,3545798427,255,1,2021-09-20,05:59:59:750,0.110178,0.099238,...,9999.99,9999.99,16399,0,224,NaN,0.109558,0.176407,-0.032841,2021-09-20 05:59:59.750
107996,DATA,1632135599,800000000,3545798442,255,1,2021-09-20,05:59:59:800,0.110141,0.099258,...,9999.99,9999.99,16399,0,227,NaN,0.106254,0.189952,-0.029897,2021-09-20 05:59:59.800
107997,DATA,1632135599,850000000,3545798457,255,1,2021-09-20,05:59:59:850,0.110127,0.099248,...,9999.99,9999.99,16399,0,112,NaN,0.127143,0.207602,-0.020573,2021-09-20 05:59:59.850
107998,DATA,1632135599,900000000,3545798472,255,1,2021-09-20,05:59:59:900,0.110135,0.099261,...,9999.99,9999.99,16399,0,215,NaN,0.108361,0.193089,-0.025130,2021-09-20 05:59:59.900


In [34]:
way3

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,Datetime-merged
0,DATA,1632130200,0,3544178502,255,1,2021-09-20,04:30:00:000,0.108009,0.100192,...,9999.99,9999.99,16399,0,98,NaN,0.136974,0.843917,-0.123542,2021-09-20 04:30:00.000
1,DATA,1632130200,50000000,3544178517,255,1,2021-09-20,04:30:00:050,0.107981,0.100206,...,9999.99,9999.99,16927,0,89,NaN,0.135590,0.826671,-0.132025,2021-09-20 04:30:00.050
2,DATA,1632130200,100000000,3544178532,255,1,2021-09-20,04:30:00:100,0.107905,0.100243,...,9999.99,9999.99,16927,0,156,NaN,0.167937,0.827212,-0.135442,2021-09-20 04:30:00.100
3,DATA,1632130200,150000000,3544178547,255,1,2021-09-20,04:30:00:150,0.107901,0.100262,...,9999.99,9999.99,16927,0,179,NaN,0.168849,0.749027,-0.147926,2021-09-20 04:30:00.150
4,DATA,1632130200,200000000,3544178562,255,1,2021-09-20,04:30:00:200,0.108036,0.100183,...,9999.99,9999.99,16927,0,166,NaN,0.236102,0.793231,-0.109815,2021-09-20 04:30:00.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,DATA,1632139199,750000000,3546878427,255,1,2021-09-20,06:59:59:750,0.130402,0.098182,...,9999.99,9999.99,15,0,23,NaN,0.074942,-0.260914,-0.016748,2021-09-20 06:59:59.750
179996,DATA,1632139199,800000000,3546878442,255,1,2021-09-20,06:59:59:800,0.130391,0.098262,...,9999.99,9999.99,15,0,242,NaN,0.092819,-0.245323,-0.012694,2021-09-20 06:59:59.800
179997,DATA,1632139199,850000000,3546878457,255,1,2021-09-20,06:59:59:850,0.130510,0.098290,...,9999.99,9999.99,15,0,241,NaN,0.075983,-0.260908,-0.016741,2021-09-20 06:59:59.850
179998,DATA,1632139199,900000000,3546878472,255,1,2021-09-20,06:59:59:900,0.130490,0.098184,...,9999.99,9999.99,15,0,85,NaN,0.076167,-0.258885,-0.012521,2021-09-20 06:59:59.900
